In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from functools import reduce
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_columns',100)

In [2]:
tbl_per_loan_card_info = pd.read_excel('ciis1112/tbl_per_loan_card_info.xlsx')

In [3]:
record_detail = pd.read_excel('ciis1112/tbl_per_recorddetail.xlsx')

In [4]:
tbl_per_info_summary = pd.read_excel('ciis1112/tbl_per_info_summary.xlsx')

In [5]:
tbl_per_loan_info = pd.read_excel('ciis1112/tbl_per_loan_info.xlsx')

In [6]:
tbl_per_person_info = pd.read_excel('ciis1112/tbl_per_personal_info.xls')
tbl_per_overdue_record = pd.read_excel('ciis1112/tbl_per_overduerecord.xls')

In [7]:
tbl_per_result_info = pd.read_excel('ciis1112/tbl_per_result_info.xls')

In [8]:
tbl_per_result_info.rename(columns={'REPORT_SN':'REPORT_ID'},inplace=True)

In [9]:
tbl_per_result_info = tbl_per_result_info[['REPORT_ID','QUERY_REASON']]

In [10]:
main_table = pd.read_excel('ciis1112/table1030_temp.xlsx')  
main_table = main_table.drop_duplicates()

In [11]:
main_table.rename(columns={'report_id':'REPORT_ID'},inplace=True)
temp1 = main_table.merge(tbl_per_result_info,on=['REPORT_ID'],how='inner')
# temp1 = temp1[temp1['QUERY_REASON'] == '贷款审批']

In [12]:
temp1 = main_table.merge(tbl_per_result_info,on=['REPORT_ID'],how='inner')
# temp1 = temp1[temp1['QUERY_REASON'] == '贷款审批']

In [13]:
dy = pd.read_excel('信贷数据调阅范围.xlsx')[['证件号码','基准时间','配偶客编','配偶证件号码']]
dy = dy.drop_duplicates()

In [14]:
dy_mr = dy.rename(columns={'证件号码':'配偶证件号码','配偶证件号码':'证件号码'})

In [15]:
temp1.rename(columns={'cert_no':'证件号码','date':'基准时间'},inplace=True)
final_main_1 = dy.merge(temp1[temp1['QUERY_REASON']=='贷款审批'],on=['证件号码','基准时间'],how='left')
final_main_1['date_delta'] = abs((final_main_1['基准时间'].astype('datetime64[ns]') - final_main_1['report_date'].astype('datetime64[ns]')).dt.days)
final_main_2 = final_main_1[final_main_1['QUERY_REASON'].isnull()][['证件号码','基准时间']]
final_main_1 = final_main_1[~final_main_1['QUERY_REASON'].isnull()].sort_values(['证件号码','date_delta'],ascending=[0,1]).groupby('证件号码').first()
final_main_2 = final_main_2.merge(temp1)
final_main_2['date_delta'] = abs((final_main_2['基准时间'].astype('datetime64[ns]') - final_main_2['report_date'].astype('datetime64[ns]')).dt.days)
final_main_2 = final_main_2.sort_values(['证件号码','date_delta'],ascending=[0,1]).groupby('证件号码').first()
final_main = final_main_1.append(final_main_2)
final_main = final_main.reset_index()
final_main.rename(columns = {'report_date':'REPORTDATE'},inplace=True)
main_table_use = final_main.copy()

In [16]:
final_main_mr_1 = dy_mr.merge(temp1[temp1['QUERY_REASON']=='贷款审批'],on=['证件号码','基准时间'],how='left')
final_main_mr_1['date_delta'] = abs((final_main_mr_1['基准时间'].astype('datetime64[ns]') - final_main_mr_1['report_date'].astype('datetime64[ns]')).dt.days)
final_main_mr_2 = final_main_mr_1[final_main_mr_1['QUERY_REASON'].isnull()][['证件号码','基准时间']]
final_main_mr_1 = final_main_mr_1[~final_main_mr_1['QUERY_REASON'].isnull()].sort_values(['证件号码','date_delta'],ascending=[0,1]).groupby('证件号码').first()
final_main_mr_2 = final_main_mr_2.merge(temp1)
final_main_mr_2['date_delta'] = abs((final_main_mr_2['基准时间'].astype('datetime64[ns]') - final_main_mr_2['report_date'].astype('datetime64[ns]')).dt.days)
final_main_mr_2 = final_main_mr_2.sort_values(['证件号码','date_delta'],ascending=[0,1]).groupby('证件号码').first()
final_main_mr = final_main_mr_1.append(final_main_mr_2)
final_main_mr = final_main_mr.reset_index()
final_main_mr.rename(columns = {'report_date':'REPORTDATE'},inplace=True)
final_main_mr = final_main_mr.dropna(subset=['REPORT_ID'])
main_table_use_mr = final_main_mr.copy()

In [17]:
final_main_mr.rename(columns={'证件号码':'配偶证件号码','配偶证件号码':'证件号码'},inplace=True)

In [18]:
# final_main.merge(final_main_mr,on=['证件号码','基准时间'],how='left').drop(['配偶客编_x','配偶证件号码_x'],axis=1)

In [19]:
# temp = pd.read_excel('loan_data/table1030_temp.xls')
# temp = temp.drop_duplicates()

In [20]:
# main_table_use_mr = main_table.rename(columns={'客户编号':'cust_no_1','cert_no':'cert_no_1','配偶客编':'客户编号',
#                                                '配偶证件号码':'cert_no'})
# main_table_use_mr = main_table_use_mr.merge(temp,on=['cert_no','date'],how='inner')
# main_table_use_mr = main_table_use_mr[~main_table_use_mr['report_id'].isnull()]
# main_table_use_mr.rename(columns={'report_id':'REPORT_ID','report_date':'REPORTDATE','date':'DATE'},inplace=True)
# # main_table_use_mr.drop('report_date',axis=1,inplace=True)

In [21]:
# main_table = main_table.merge(temp,on=['cert_no','date'],how='left')
# main_table.rename(columns={'report_id':'REPORT_ID','report_date':'REPORTDATE','date':'DATE'},inplace=True)
# # main_table.drop('report_date',axis=1,inplace=True)
# main_table_use = main_table.copy()
# main_table_use = main_table_use[~main_table_use['REPORT_ID'].isnull()]

In [22]:
all_merge = lambda x,y: x.merge(y,on=['REPORT_ID'],how='left')

## 1.累计逾期与最大逾期（近3月，近6月，近12月，近24月）

In [23]:
def cumsum_overdue(s):
    fetch_list = re.findall('(\d)',s)
    return len(fetch_list)

def max_overdue(s):
    fetch_list = re.findall('(\d)',s)
    if len(fetch_list) < 1:
        return 0
    else:
        return int(max(fetch_list))

### 1.1 普通贷款逾期数据

In [24]:
overdue_data_1 = tbl_per_loan_info[['REPORT_ID','LATEST24STATE','LAST24MONTHBEGINMONTH','LAST24MONTHENDMONTH']]
date_range_24_1 = overdue_data_1.groupby('REPORT_ID')[['LAST24MONTHBEGINMONTH','LAST24MONTHENDMONTH']].first()
overdue_data_1 = overdue_data_1.dropna(subset=['LATEST24STATE'])
overdue_data_1['LATEST3STATE'] = overdue_data_1['LATEST24STATE'].str.split(',').apply(lambda x:','.join(x[-4:]))
overdue_data_1['LATEST6STATE'] = overdue_data_1['LATEST24STATE'].str.split(',').apply(lambda x:','.join(x[-7:]))
overdue_data_1['LATEST12STATE'] = overdue_data_1['LATEST24STATE'].str.split(',').apply(lambda x:','.join(x[-13:]))
overdue_data_1['max_overdue_3'] = overdue_data_1['LATEST3STATE'].apply(max_overdue)
overdue_data_1['cumsum_overdue_3'] = overdue_data_1['LATEST3STATE'].apply(cumsum_overdue)
overdue_data_1['max_overdue_6'] = overdue_data_1['LATEST6STATE'].apply(max_overdue)
overdue_data_1['cumsum_overdue_6'] = overdue_data_1['LATEST6STATE'].apply(cumsum_overdue)
overdue_data_1['max_overdue_12'] = overdue_data_1['LATEST12STATE'].apply(max_overdue)
overdue_data_1['cumsum_overdue_12'] = overdue_data_1['LATEST12STATE'].apply(cumsum_overdue)
overdue_data_1['max_overdue_24'] = overdue_data_1['LATEST24STATE'].apply(max_overdue)
overdue_data_1['cumsum_overdue_24'] = overdue_data_1['LATEST24STATE'].apply(cumsum_overdue)
overdue_data_1_3 = overdue_data_1.groupby('REPORT_ID').agg({'max_overdue_3':'max','cumsum_overdue_3':'sum'})
overdue_data_1_6 = overdue_data_1.groupby('REPORT_ID').agg({'max_overdue_6':'max','cumsum_overdue_6':'sum'})
overdue_data_1_12 = overdue_data_1.groupby('REPORT_ID').agg({'max_overdue_12':'max','cumsum_overdue_12':'sum'})
overdue_data_1_24 = overdue_data_1.groupby('REPORT_ID').agg({'max_overdue_24':'max','cumsum_overdue_24':'sum'})
# cumsum_overdue_24['CUMSUM_OVERDUE_24'] = cumsum_overdue_24['LATEST24STATE'].apply(max_overdue)

### 1.2 贷记卡贷款逾期数据

In [25]:
overdue_data_2 = tbl_per_loan_card_info[['REPORT_ID','LAST_24STATE','BEGINMONTH_24MONTH','ENDMONTH_24MONTH']]
# date_range_24_2 = overdue_data_2.groupby('REPORT_ID')[['BEGINMONTH_24MONTH','ENDMONTH_24MONTH']].first()
overdue_data_2 = overdue_data_2.dropna(subset=['LAST_24STATE'])
overdue_data_2['LATEST3STATE'] = overdue_data_2['LAST_24STATE'].str.split(',').apply(lambda x:','.join(x[-4:]))
overdue_data_2['LATEST6STATE'] = overdue_data_2['LAST_24STATE'].str.split(',').apply(lambda x:','.join(x[-7:]))
overdue_data_2['LATEST12STATE'] = overdue_data_2['LAST_24STATE'].str.split(',').apply(lambda x:','.join(x[-13:]))
overdue_data_2['max_overdue_3'] = overdue_data_2['LATEST3STATE'].apply(max_overdue)
overdue_data_2['cumsum_overdue_3'] = overdue_data_2['LATEST3STATE'].apply(cumsum_overdue)
overdue_data_2['max_overdue_6'] = overdue_data_2['LATEST6STATE'].apply(max_overdue)
overdue_data_2['cumsum_overdue_6'] = overdue_data_2['LATEST6STATE'].apply(cumsum_overdue)
overdue_data_2['max_overdue_12'] = overdue_data_2['LATEST12STATE'].apply(max_overdue)
overdue_data_2['cumsum_overdue_12'] = overdue_data_2['LATEST12STATE'].apply(cumsum_overdue)
overdue_data_2['max_overdue_24'] = overdue_data_2['LAST_24STATE'].apply(max_overdue)
overdue_data_2['cumsum_overdue_24'] = overdue_data_2['LAST_24STATE'].apply(cumsum_overdue)
overdue_data_2_3 = overdue_data_2.groupby('REPORT_ID').agg({'max_overdue_3':'max','cumsum_overdue_3':'sum'})
overdue_data_2_6 = overdue_data_2.groupby('REPORT_ID').agg({'max_overdue_6':'max','cumsum_overdue_6':'sum'})
overdue_data_2_12 = overdue_data_2.groupby('REPORT_ID').agg({'max_overdue_12':'max','cumsum_overdue_12':'sum'})
overdue_data_2_24 = overdue_data_2.groupby('REPORT_ID').agg({'max_overdue_24':'max','cumsum_overdue_24':'sum'})

### 1.3 已结清累计逾期特征（近5年）

In [26]:
overdue_record = tbl_per_overdue_record[['REPORT_ID','LASTMONTHS','MONTH']]
overdue_record_hs = overdue_record.groupby('REPORT_ID').agg({'LASTMONTHS':['max','sum']})
overdue_record_hs = overdue_record_hs['LASTMONTHS'].reset_index()
overdue_record_hs.columns = ['REPORT_ID','近5年已结清最大逾期','近5年已结清累计逾期']

In [27]:
# tbl_per_overdue_record['MONTH'].astype(str).apply(lambda x:''.join(x.split('.'
def minus_month_6(x):
    if x.month <= 6:
        mins = x.month - 6
        month = 12+mins
        return x.replace(x.year-1,month,1)
    else:
        return x.replace(x.year,x.month-6,1)
    
def minus_month_3(x):
    if x.month <= 3:
        mins = x.month - 3
        month = 12+mins
        return x.replace(x.year-1,month,1)
    else:
        return x.replace(x.year,x.month-3,1)
    
overdue_data_3 = overdue_record.merge(date_range_24_1.reset_index())
overdue_data_3 = overdue_data_3.dropna(subset=['MONTH','LAST24MONTHBEGINMONTH','LAST24MONTHENDMONTH'])
overdue_data_3['MONTH'] = pd.to_datetime(overdue_data_3['MONTH'].astype(str).apply(lambda x: x[:4]+'-'+x[5:]))
overdue_data_3['LAST24MONTHBEGINMONTH'] = pd.to_datetime(overdue_data_3['LAST24MONTHBEGINMONTH'].astype(str).apply(lambda x: x[:4]+'-'+x[5:]))
overdue_data_3['LAST24MONTHENDMONTH'] = pd.to_datetime(overdue_data_3['LAST24MONTHENDMONTH'].astype(str).apply(lambda x: x[:4]+'-'+x[5:]))
overdue_data_3['LAST3MONTHBEGINMONTH'] = overdue_data_3['LAST24MONTHENDMONTH'].apply(minus_month_3)
overdue_data_3['LAST6MONTHBEGINMONTH'] = overdue_data_3['LAST24MONTHENDMONTH'].apply(minus_month_6)
overdue_data_3['LAST12MONTHBEGINMONTH'] = overdue_data_3['LAST24MONTHENDMONTH'].apply(lambda x: x.replace(x.year-1,x.month))
overdue_data_3_24 = overdue_data_3[(overdue_data_3['MONTH']>=overdue_data_3['LAST24MONTHBEGINMONTH']
                                   )&(overdue_data_3['MONTH']<=overdue_data_3['LAST24MONTHENDMONTH']
                                     )]
overdue_data_3_12 = overdue_data_3[overdue_data_3['MONTH']>=overdue_data_3['LAST12MONTHBEGINMONTH']]
overdue_data_3_6 = overdue_data_3[overdue_data_3['MONTH']>=overdue_data_3['LAST6MONTHBEGINMONTH']]
overdue_data_3_3 = overdue_data_3[overdue_data_3['MONTH']>=overdue_data_3['LAST3MONTHBEGINMONTH']]
overdue_data_3_24 = overdue_data_3_24.groupby('REPORT_ID').agg({'LASTMONTHS':['sum','max']}).reset_index()
overdue_data_3_24.columns = ['REPORT_ID','max_overdue_24','cumsum_overdue_24']
overdue_data_3_12 = overdue_data_3_12.groupby('REPORT_ID').agg({'LASTMONTHS':['max','sum']}).reset_index()
overdue_data_3_12.columns = ['REPORT_ID','max_overdue_12','cumsum_overdue_12']
overdue_data_3_6 = overdue_data_3_6.groupby('REPORT_ID').agg({'LASTMONTHS':['max','sum']}).reset_index()
overdue_data_3_6.columns = ['REPORT_ID','max_overdue_6','cumsum_overdue_6']
overdue_data_3_3 = overdue_data_3_3.groupby('REPORT_ID').agg({'LASTMONTHS':['max','sum']}).reset_index()
overdue_data_3_3.columns = ['REPORT_ID','max_overdue_3','cumsum_overdue_3']

In [28]:
overdue_data_2_all = reduce(lambda x,y: x.merge(y,on='REPORT_ID',how='outer'),[overdue_data_2_3,overdue_data_2_6,overdue_data_2_12,overdue_data_2_24])
overdue_data_1_all = reduce(lambda x,y: x.merge(y,on='REPORT_ID',how='outer'),[overdue_data_1_3,overdue_data_1_6,overdue_data_1_12,overdue_data_1_24])
overdue_data_3_all = reduce(lambda x,y: x.merge(y,on='REPORT_ID',how='outer'),[overdue_data_3_3,overdue_data_3_6,overdue_data_3_12,overdue_data_3_24])

In [29]:
overdue_data_3_all = overdue_data_3_all.set_index('REPORT_ID')

In [30]:
overdue_data_all = overdue_data_1_all.merge(overdue_data_2_all,on='REPORT_ID',how='outer')
overdue_data_all = overdue_data_all.merge(overdue_data_3_all,on='REPORT_ID',how='outer')
for c in overdue_data_1_all.columns:
    cols = overdue_data_all.columns
    if 'max' in c:
        overdue_data_all[c] = overdue_data_all[cols[cols.str.contains(c)]].max(axis=1)
    else:
        overdue_data_all[c] = overdue_data_all[cols[overdue_data_all.columns.str.contains(c)]].sum(axis=1)
overdue_data_all = overdue_data_all.iloc[:,-8:]

In [31]:
overdue_data_all.columns = ['近3月最大逾期','近3月累计逾期','近6月最大逾期','近6月累计逾期','近12月最大逾期','近12月累计逾期','近24月最大逾期','近24月累计逾期']
overdue_data_all = overdue_data_all.reset_index()

In [32]:
overdue_data_all.head()

,REPORT_ID,近3月最大逾期,近3月累计逾期,近6月最大逾期,近6月累计逾期,近12月最大逾期,近12月累计逾期,近24月最大逾期,近24月累计逾期
0,2018111300006460106371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018111300006460120325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018111300006460126410,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
3,2018111300006460193304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018111300006460216344,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


## 2.查询次数与查询机构数（担保审批、贷款审批、信用卡审批）

In [33]:
cur_date = datetime.datetime.today()

In [34]:
def minus_month_6(x):
    if x.month <= 6:
        mins = x.month - 6
        month = 12+mins
        return x.replace(x.year-1,month,1)
    else:
        return x.replace(x.year,x.month-6,1)

def minus_month_2(x):
    if x.month <= 2:
        mins = x.month - 2
        month = 12+mins
        return x.replace(x.year-1,month,1)
    else:
        return x.replace(x.year,x.month-2,1)
    
record_data = record_detail[['REPORT_ID','QUERIER','QUERYREASON','QUERYDATE']]
record_data['QUERIER'] = record_data['QUERIER'].astype(str)
record_data.loc[record_data['QUERIER'].str.contains('厦门国际银行'),'QUERIER'] = '厦门国际银行'
record_data = record_data.merge(main_table_use[['REPORT_ID','REPORTDATE']],on=['REPORT_ID'])
record_data['REPORTDATE'] = record_data['REPORTDATE'].astype(str).apply(lambda x:'-'.join([x[:4],x[5:7],x[8:10]]))
record_data['REPORTDATE'] = pd.to_datetime(record_data['REPORTDATE'])
record_data['QUERYDATE'] = record_data['QUERYDATE'].astype(str).apply(lambda x:'-'.join([x[:4],x[5:7],x[8:10]]))
record_data['QUERYDATE'] = pd.to_datetime(record_data['QUERYDATE'])
record_data['LAST24DATE'] = record_data['REPORTDATE'].apply(lambda x:x.replace(x.year-2))
record_data['LAST12DATE'] = record_data['REPORTDATE'].apply(lambda x:x.replace(x.year-1))
record_data['LAST6DATE'] = record_data['REPORTDATE'].apply(minus_month_6)
record_data['LAST2DATE'] = record_data['REPORTDATE'].apply(minus_month_2)
record_data = record_data[~record_data['QUERIER'].str.contains('厦门国际银行')]
record_data_24 = record_data[record_data['QUERYDATE'] >= record_data['LAST24DATE']]
record_data_12 = record_data[record_data['QUERYDATE'] >= record_data['LAST12DATE']]
record_data_6 = record_data[record_data['QUERYDATE'] >= record_data['LAST6DATE']]
record_data_2 = record_data[record_data['QUERYDATE'] >= record_data['LAST2DATE']]
lastquerydate = record_data_12.groupby('REPORT_ID')[['QUERYDATE']].first()
lastquerydate['LASTQUERYMONTHS'] = lastquerydate['QUERYDATE'].apply(lambda x:round((cur_date-x).days/30))
lastquerydate = lastquerydate.reset_index()
del lastquerydate['QUERYDATE']
record_org_count_24 = record_data_24.groupby(['REPORT_ID','QUERYREASON'])['QUERIER'].apply(lambda x:len(x.unique())).unstack()
record_count_24 = record_data_24.groupby(['REPORT_ID','QUERYREASON'])['QUERIER'].count().unstack()
record_org_count_12 = record_data_12.groupby(['REPORT_ID','QUERYREASON'])['QUERIER'].apply(lambda x:len(x.unique())).unstack()
record_count_12 = record_data_12.groupby(['REPORT_ID','QUERYREASON'])['QUERIER'].count().unstack()
record_org_count_6 = record_data_6.groupby(['REPORT_ID','QUERYREASON'])['QUERIER'].apply(lambda x:len(x.unique())).unstack()
record_count_6 = record_data_6.groupby(['REPORT_ID','QUERYREASON'])['QUERIER'].count().unstack()
record_org_count_2 = record_data_2.groupby(['REPORT_ID','QUERYREASON'])['QUERIER'].apply(lambda x:len(x.unique())).unstack()
record_count_2 = record_data_2.groupby(['REPORT_ID','QUERYREASON'])['QUERIER'].count().unstack()
record_org_count_24 = record_org_count_24.add_prefix('近2年内查询机构数-')
record_count_24 = record_count_24.add_prefix('近2年内查询次数-')
record_org_count_12 = record_org_count_12.add_prefix('近12月内查询机构数-')
record_count_12 = record_count_12.add_prefix('近12月内查询次数-')
record_org_count_6 = record_org_count_6.add_prefix('近6月内查询机构数-')
record_count_6 = record_count_6.add_prefix('近6月内查询次数-')
record_org_count_2 = record_org_count_2.add_prefix('近2月内查询机构数-')
record_count_2 = record_count_2.add_prefix('近2月内查询次数-')
record_all = reduce(lambda x,y:x.merge(y,on='REPORT_ID',how='outer'),[record_count_2,record_org_count_2,record_count_6,record_org_count_6,
                                                                      record_count_12,record_org_count_12,record_count_24,
                                                                      record_org_count_24,lastquerydate])
record_all.rename(columns={'LASTQUERYMONTHS':'十二个月内最近一次查询距今月份'},inplace=True)

<ipython-input-34-2dabaaf75cad>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  record_data['QUERIER'] = record_data['QUERIER'].astype(str)
D:\tools\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-34-2dabaaf75cad>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec

In [35]:
record_all.head()

,REPORT_ID,近2月内查询次数-信用卡审批,近2月内查询次数-担保资格审查,近2月内查询次数-贷款审批,近2月内查询机构数-信用卡审批,近2月内查询机构数-担保资格审查,近2月内查询机构数-贷款审批,近6月内查询次数-信用卡审批,近6月内查询次数-担保资格审查,近6月内查询次数-贷款审批,近6月内查询机构数-信用卡审批,近6月内查询机构数-担保资格审查,近6月内查询机构数-贷款审批,近12月内查询次数-信用卡审批,近12月内查询次数-担保资格审查,近12月内查询次数-贷款审批,近12月内查询机构数-信用卡审批,近12月内查询机构数-担保资格审查,近12月内查询机构数-贷款审批,近2年内查询次数-信用卡审批,近2年内查询次数-担保资格审查,近2年内查询次数-贷款审批,近2年内查询机构数-信用卡审批,近2年内查询机构数-担保资格审查,近2年内查询机构数-贷款审批,十二个月内最近一次查询距今月份
0,2018111300006460193304,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,4.0,26.0
1,2018111300006460216344,1.0,NaN,2.0,1.0,NaN,2.0,1.0,NaN,2.0,1.0,NaN,2.0,1.0,NaN,2.0,1.0,NaN,2.0,7.0,NaN,3.0,5.0,NaN,3.0,24.0
2,2018111300006460324975,NaN,NaN,3.0,NaN,NaN,1.0,NaN,1.0,9.0,NaN,1.0,6.0,NaN,1.0,13.0,NaN,1.0,7.0,NaN,1.0,16.0,NaN,1.0,10.0,25.0
3,2018111300006460325386,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,2.0,NaN,1.0,2.0,NaN,1.0,4.0,NaN,5.0,4.0,NaN,4.0,25.0
4,2018111300006460360719,NaN,2.0,1.0,NaN,1.0,1.0,1.0,3.0,4.0,1.0,1.0,4.0,3.0,6.0,9.0,3.0,1.0,9.0,6.0,10.0,13.0,5.0,1.0,11.0,25.0


## 3.非银机构特征

In [36]:
tbl_per_loan_info.columns[tbl_per_loan_info.columns.str.contains('ORG')]

Index(['FINANCEORG', 'LOAN_ORG'], dtype='object')

In [37]:
exp_bank_data = tbl_per_loan_info[['REPORT_ID','STATE','CUE','LOAN_ORG']]
exp_bank_data = exp_bank_data[~exp_bank_data['CUE'].str.contains('银行')]
exp_bank_org_count = exp_bank_data.groupby('REPORT_ID').apply(lambda x: len(x['LOAN_ORG'].unique()))
exp_bank_no_settle = exp_bank_data[(exp_bank_data['STATE'] != '结清')].groupby('REPORT_ID').agg({'STATE':'count'})
exp_bank_data_all = pd.concat([exp_bank_org_count,exp_bank_no_settle],axis=1)
exp_bank_data_all.columns=['非银金融机构信贷历史机构数','非银金融机构未结清贷款笔数']
exp_bank_data_all = exp_bank_data_all.reset_index()
exp_bank_data_all = exp_bank_data_all.rename(columns={'index':'REPORT_ID'})

In [38]:
exp_bank_data_all.head()

,REPORT_ID,非银金融机构信贷历史机构数,非银金融机构未结清贷款笔数
0,2018111300006460106371,1,1.0
1,2018111300006460193304,1,1.0
2,2018111300006460360719,1,NaN
3,2018111300006460590451,3,2.0
4,2018111300006460599541,2,1.0


## 4.近6月平均使用额度、发卡法人机构总数、贷记卡平均额度、贷记卡最高额度、贷记卡透支余额汇总、贷记卡最高透支余额

In [39]:
used_credit = tbl_per_info_summary[['REPORT_ID','UNDESTYRYLOANCARD_USED_AVG_6','UNDESTYRYLOANCARD_USED_CREDIT',
                                    'UNDESTYRYLOANCARD_CREDIT_LIMIT','GUAR_BALANCE','UNDESTYRYSLOANCARD_MAX_CREDIT']]
used_credit.columns = ['REPORT_ID','近6月平均使用额度','透支余额汇总','授信总额','担保本金余额','单家行最高授信总额']

In [40]:
used_credit['发卡法人机构总数'] = tbl_per_info_summary[['UNDESTYRYLOANCARD_FANC_COUNT','UNDESTYRYSLOANCARD_FANC_COUNT']].fillna(0).sum(axis=1)
used_credit['平均额度'] = tbl_per_info_summary['UNDESTYRYLOANCARD_CREDIT_LIMIT']/tbl_per_info_summary['UNDESTYRYLOANCARD_FANC_COUNT']

<ipython-input-40-a37ecebab48c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_credit['发卡法人机构总数'] = tbl_per_info_summary[['UNDESTYRYLOANCARD_FANC_COUNT','UNDESTYRYSLOANCARD_FANC_COUNT']].fillna(0).sum(axis=1)
<ipython-input-40-a37ecebab48c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_credit['平均额度'] = tbl_per_info_summary['UNDESTYRYLOANCARD_CREDIT_LIMIT']/tbl_per_info_summary['UNDESTYRYLOANCARD_FANC_COUNT']


## 5.贷记透支比例(额度总使用率)

In [41]:
used_credit['贷记透支比例'] = used_credit['透支余额汇总']/used_credit['授信总额']
used_credit = used_credit.fillna(0)

<ipython-input-41-1b59797ae44d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_credit['贷记透支比例'] = used_credit['透支余额汇总']/used_credit['授信总额']


## 6.对外担保余额

In [42]:
guar_balance = used_credit.groupby('REPORT_ID',as_index=False)['担保本金余额'].sum()
guar_balance.columns = ['REPORT_ID','对外担保余额']

In [43]:
guar_balance.head()

,REPORT_ID,对外担保余额
0,2018111300006460106371,0.0
1,2018111300006460120325,0.0
2,2018111300006460126410,7000000.0
3,2018111300006460193304,330665.0
4,2018111300006460198247,0.0


## 7.未结清贷款笔数，未结清贷款余额

In [44]:
no_settle_info = tbl_per_loan_info[['REPORT_ID','STATE','BALANCE']]
no_settle_info = no_settle_info[no_settle_info['STATE']!='结清']
no_settle_info = no_settle_info.groupby('REPORT_ID',as_index=False).agg({'BALANCE':['count','sum']})
no_settle_info.columns = ['REPORT_ID','未结清贷款笔数','未结清贷款余额']

In [45]:
no_settle_info.head()

,REPORT_ID,未结清贷款笔数,未结清贷款余额
0,2018111300006460106371,81,5309813.0
1,2018111300006460120325,1,7000000.0
2,2018111300006460126410,1,1850000.0
3,2018111300006460193304,2,187381.0
4,2018111300006460216344,1,1000000.0


## 8.贷款历史机构数

In [46]:
his_loan_org_count = tbl_per_loan_info[['REPORT_ID','LOAN_ORG']]
his_loan_org_count = his_loan_org_count.groupby('REPORT_ID').apply(lambda x:len(x['LOAN_ORG'].unique())).reset_index()
his_loan_org_count.columns = ['REPORT_ID','贷款历史机构数']

In [47]:
his_loan_org_count.head()

,REPORT_ID,贷款历史机构数
0,2018111300006460106371,4
1,2018111300006460120325,2
2,2018111300006460126410,2
3,2018111300006460193304,2
4,2018111300006460216344,4


## 9.贷款总额，贷款总笔数

In [48]:
total_loan_info = tbl_per_info_summary[['REPORT_ID','UNPAIDLOAN_ACCOUNT_COUNT','UNPAIDLOAN_BALANCE']]
total_loan_info = total_loan_info.groupby('REPORT_ID',as_index=False).sum()
total_loan_info.columns = ['REPORT_ID','贷款总笔数','贷款总额']

## <font color=red face="微软雅黑">10.存在余额开户数、存在余额六个月内开户数、存在余额十二个月内开户数?</font>

In [49]:
per_no_zero_accont = tbl_per_loan_info[tbl_per_loan_info['BALANCE'] != 0][['REPORT_ID','LOAN_START']]

In [50]:
tbl_per_info_summary[tbl_per_info_summary['REPORT_ID'] == '2018101200006278567160'][['REPORT_ID','UNDESTYRYLOANCARD_ACCOUNTCOUN',
                                                                                     'UNDESTYRYSLOANCARD_ACCOUNTCOUN']]

,REPORT_ID,UNDESTYRYLOANCARD_ACCOUNTCOUN,UNDESTYRYSLOANCARD_ACCOUNTCOUN


## 11.贷款最大剩余月数、贷款最大还款月数、其他贷款笔数占比、住房贷款笔数占比

In [51]:
pay_info = tbl_per_loan_info[['REPORT_ID','REMAINPAYMENTCYC','CUE','TYPE','STATE']]
# remain_pay_max = pay_info.groupby('REPORT_ID',as_index=False)[['REMAINPAYMENTCYC']].max()
pay_info['NEEDPAYMONTHS'] = pay_info['CUE'].str.extract('，(\d+)期').fillna(0).astype(int)
pay_info['HAVEPAY'] = pay_info['NEEDPAYMONTHS']-pay_info['REMAINPAYMENTCYC']
pay_max = pay_info.groupby('REPORT_ID',as_index=False).agg({'REMAINPAYMENTCYC':'max','HAVEPAY':'max'})
pay_max.columns = ['REPORT_ID','贷款最大剩余月数','贷款最大还款月数']

other_loan_count = pay_info[(pay_info['TYPE'] == '其他贷款')&(pay_info['STATE'] != '结清')].groupby('REPORT_ID',as_index=False)[['TYPE']].count()
other_loan_ratio = other_loan_count.merge(total_loan_info,how='outer')
other_loan_ratio['其他贷款占比'] = other_loan_ratio['TYPE']/other_loan_ratio['贷款总笔数']
other_loan_ratio.drop(['贷款总额','贷款总笔数','TYPE'],axis=1,inplace=True)

resid_loan_count = pay_info[(pay_info['TYPE'] == '个人住房贷款')&(pay_info['STATE'] != '结清')].groupby('REPORT_ID',as_index=False)[['TYPE']].count()
resid_loan_ratio = resid_loan_count.merge(total_loan_info,how='outer')
resid_loan_ratio['住房贷款占比'] = resid_loan_ratio['TYPE']/resid_loan_ratio['贷款总笔数']
resid_loan_ratio.drop(['贷款总额','贷款总笔数','TYPE'],axis=1,inplace=True)

pay_and_loan_ratio = reduce(lambda x,y:x.merge(y),[pay_max,other_loan_ratio,resid_loan_ratio])
pay_and_loan_ratio.head()

<ipython-input-51-f2435d4223a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pay_info['NEEDPAYMONTHS'] = pay_info['CUE'].str.extract('，(\d+)期').fillna(0).astype(int)
<ipython-input-51-f2435d4223a0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pay_info['HAVEPAY'] = pay_info['NEEDPAYMONTHS']-pay_info['REMAINPAYMENTCYC']


,REPORT_ID,贷款最大剩余月数,贷款最大还款月数,其他贷款占比,住房贷款占比
0,2018111300006460106371,175.0,65.0,NaN,0.012346
1,2018111300006460120325,1.0,12.0,NaN,NaN
2,2018111300006460126410,24.0,12.0,NaN,NaN
3,2018111300006460193304,26.0,10.0,0.5,NaN
4,2018111300006460216344,2.0,11.0,NaN,NaN


## 12.征信贷记卡最近一次M0距今月数、贷记卡最近一次M1及以上距今月数、贷记卡最近一次M2及以  上距今月数、贷记卡最近一次M3及以上距今月数、贷记卡最近一次M4及以上距今月数

In [52]:
overdue_data_loan_card = tbl_per_loan_card_info[['REPORT_ID','LAST_24STATE','BEGINMONTH_24MONTH','ENDMONTH_24MONTH']]
overdue_data_loan_card = overdue_data_loan_card.dropna(subset=['LAST_24STATE'])

In [53]:
import re

def check_index_m0(x):
    try:
        sparray = np.array(x.split(','))
        a = 24-(np.where(sparray=='1')[0][-1]+1)
    except:
        a = np.nan
    return a

def check_index_m1(x):
    try:
        last_num = re.findall('(\d)',x)[-1]
        sparray = np.array(x.split(','))
        a = 24-(np.where(sparray==last_num)[0][-1]+1)
    except:
        a = np.nan
    return a

def check_index_m2(x):
    try:
        over_array = np.array(re.findall('(\d)',x)).astype(int)
        last_num = str(over_array[over_array>1][-1])
        sparray = np.array(x.split(','))
        a = 24-(np.where(sparray==last_num)[0][-1]+1)
    except:
        a = np.nan
    return a

def check_index_m3(x):
    try:
        over_array = np.array(re.findall('(\d)',x)).astype(int)
        last_num = str(over_array[over_array>2][-1])
        sparray = np.array(x.split(','))
        a = 24-(np.where(sparray==last_num)[0][-1]+1)
    except:
        a = np.nan
    return a

def check_index_m4(x):
    try:
        over_array = np.array(re.findall('(\d)',x)).astype(int)
        last_num = str(over_array[over_array>3][-1])
        sparray = np.array(x.split(','))
        a = 24-(np.where(sparray==last_num)[0][-1]+1)
    except:
        a = np.nan
    return a

overdue_data_loan_card['贷记卡最近一次M0距今月数'] = overdue_data_loan_card['LAST_24STATE'].apply(check_index_m0)
overdue_data_loan_card['贷记卡最近一次M1及以上距今月数'] = overdue_data_loan_card['LAST_24STATE'].apply(check_index_m1)
overdue_data_loan_card['贷记卡最近一次M2及以上距今月数'] = overdue_data_loan_card['LAST_24STATE'].apply(check_index_m2)
overdue_data_loan_card['贷记卡最近一次M3及以上距今月数'] = overdue_data_loan_card['LAST_24STATE'].apply(check_index_m3)
overdue_data_loan_card['贷记卡最近一次M4及以上距今月数'] = overdue_data_loan_card['LAST_24STATE'].apply(check_index_m4)

In [54]:
accage_loan_card = overdue_data_loan_card.groupby('REPORT_ID',as_index=False)[['贷记卡最近一次M0距今月数','贷记卡最近一次M1及以上距今月数',
                                                             '贷记卡最近一次M2及以上距今月数','贷记卡最近一次M3及以上距今月数',
                                                             '贷记卡最近一次M4及以上距今月数']].min()
accage_loan_card.head()

,REPORT_ID,贷记卡最近一次M0距今月数,贷记卡最近一次M1及以上距今月数,贷记卡最近一次M2及以上距今月数,贷记卡最近一次M3及以上距今月数,贷记卡最近一次M4及以上距今月数
0,2018111300006460106371,NaN,NaN,NaN,NaN,NaN
1,2018111300006460120325,NaN,NaN,NaN,NaN,NaN
2,2018111300006460126410,1.0,1.0,NaN,NaN,NaN
3,2018111300006460193304,NaN,NaN,NaN,NaN,NaN
4,2018111300006460216344,3.0,3.0,NaN,NaN,NaN


## 13.贷款三个月内M1次数、贷款三个月内M2次数、贷款三个月内M3次数、贷款三个月内M4次数、贷款三个月内M7次数

In [55]:
overdue_data_loan = tbl_per_loan_info[['REPORT_ID','LATEST24STATE','LAST24MONTHBEGINMONTH','LAST24MONTHENDMONTH']]
overdue_data_loan = overdue_data_loan.dropna(subset=['LATEST24STATE'])
overdue_data_loan['LATEST3STATE'] = overdue_data_loan['LATEST24STATE'].str.split(',').apply(lambda x:','.join(x[-4:]))
overdue_data_loan['LATEST6STATE'] = overdue_data_loan['LATEST24STATE'].str.split(',').apply(lambda x:','.join(x[-7:]))
overdue_data_loan['LATEST12STATE'] = overdue_data_loan['LATEST24STATE'].str.split(',').apply(lambda x:','.join(x[-13:]))

In [56]:
overdue_data_loan['贷款三个月内M1次数'] = overdue_data_loan['LATEST3STATE'].str.findall('(1)').apply(lambda x:len(x))
overdue_data_loan['贷款三个月内M2次数'] = overdue_data_loan['LATEST3STATE'].str.findall('(2)').apply(lambda x:len(x))
overdue_data_loan['贷款三个月内M3次数'] = overdue_data_loan['LATEST3STATE'].str.findall('(3)').apply(lambda x:len(x))
overdue_data_loan['贷款三个月内M4次数'] = overdue_data_loan['LATEST3STATE'].str.findall('(4)').apply(lambda x:len(x))
overdue_data_loan['贷款三个月内M7次数'] = overdue_data_loan['LATEST3STATE'].str.findall('(7)').apply(lambda x:len(x))

In [57]:
overdue_data_loan['贷款六个月内M1次数'] = overdue_data_loan['LATEST6STATE'].str.findall('(1)').apply(lambda x:len(x))
overdue_data_loan['贷款六个月内M2次数'] = overdue_data_loan['LATEST6STATE'].str.findall('(2)').apply(lambda x:len(x))
overdue_data_loan['贷款六个月内M3次数'] = overdue_data_loan['LATEST6STATE'].str.findall('(3)').apply(lambda x:len(x))
overdue_data_loan['贷款六个月内M4次数'] = overdue_data_loan['LATEST6STATE'].str.findall('(4)').apply(lambda x:len(x))
overdue_data_loan['贷款六个月内M7次数'] = overdue_data_loan['LATEST6STATE'].str.findall('(7)').apply(lambda x:len(x))

In [58]:
overdue_data_loan['贷款十二个月内M1次数'] = overdue_data_loan['LATEST12STATE'].str.findall('(1)').apply(lambda x:len(x))
overdue_data_loan['贷款十二个月内M2次数'] = overdue_data_loan['LATEST12STATE'].str.findall('(2)').apply(lambda x:len(x))
overdue_data_loan['贷款十二个月内M3次数'] = overdue_data_loan['LATEST12STATE'].str.findall('(3)').apply(lambda x:len(x))
overdue_data_loan['贷款十二个月内M4次数'] = overdue_data_loan['LATEST12STATE'].str.findall('(4)').apply(lambda x:len(x))
overdue_data_loan['贷款十二个月内M7次数'] = overdue_data_loan['LATEST12STATE'].str.findall('(7)').apply(lambda x:len(x))

In [59]:
cols = overdue_data_loan.columns
accage_count_loan = overdue_data_loan.groupby('REPORT_ID',as_index=False)[cols[cols.str.contains('次数')]].sum()

In [60]:
accage_count_loan.head()

,REPORT_ID,贷款三个月内M1次数,贷款三个月内M2次数,贷款三个月内M3次数,贷款三个月内M4次数,贷款三个月内M7次数,贷款六个月内M1次数,贷款六个月内M2次数,贷款六个月内M3次数,贷款六个月内M4次数,贷款六个月内M7次数,贷款十二个月内M1次数,贷款十二个月内M2次数,贷款十二个月内M3次数,贷款十二个月内M4次数,贷款十二个月内M7次数
0,2018111300006460106371,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018111300006460120325,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018111300006460126410,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2018111300006460193304,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2018111300006460216344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 14.征信贷款最近一次M0距今月数、征信贷款最近一次M1及以上距今月数、征信贷款最近一次M2及以上距今月数、征信贷款最近一次M3及以上距今月数、征信贷款最近一次M4及以上距今月数

In [61]:
overdue_data_loan['贷款最近一次M0距今月数'] = overdue_data_loan['LATEST24STATE'].apply(check_index_m0)
overdue_data_loan['贷款最近一次M1及以上距今月数'] = overdue_data_loan['LATEST24STATE'].apply(check_index_m1)
overdue_data_loan['贷款最近一次M2及以上距今月数'] = overdue_data_loan['LATEST24STATE'].apply(check_index_m2)
overdue_data_loan['贷款最近一次M3及以上距今月数'] = overdue_data_loan['LATEST24STATE'].apply(check_index_m3)
overdue_data_loan['贷款最近一次M4及以上距今月数'] = overdue_data_loan['LATEST24STATE'].apply(check_index_m4)
# overdue_data_loan = overdue_data_loan.fillna(0)
accage_loan = overdue_data_loan.groupby('REPORT_ID',as_index=False)[['贷款最近一次M0距今月数','贷款最近一次M1及以上距今月数',
                                                             '贷款最近一次M2及以上距今月数','贷款最近一次M3及以上距今月数',
                                                             '贷款最近一次M4及以上距今月数']].min()

In [62]:
accage_loan.head()

,REPORT_ID,贷款最近一次M0距今月数,贷款最近一次M1及以上距今月数,贷款最近一次M2及以上距今月数,贷款最近一次M3及以上距今月数,贷款最近一次M4及以上距今月数
0,2018111300006460106371,NaN,NaN,NaN,NaN,NaN
1,2018111300006460120325,NaN,NaN,NaN,NaN,NaN
2,2018111300006460126410,NaN,NaN,NaN,NaN,NaN
3,2018111300006460193304,NaN,NaN,NaN,NaN,NaN
4,2018111300006460216344,NaN,NaN,NaN,NaN,NaN


## 15.性别、年龄、婚姻状况、学历

In [63]:
personal_data = tbl_per_person_info[['REPORT_ID','GENDER','BIRTHDAY','MARITAL_STSTE','EDU_LEVEL']]
personal_data['GENDER'] = personal_data['GENDER'].replace({'男性':1,'女性':0,'--':0})
cur_date = datetime.datetime.today()
personal_data['BIRTHDAY'] = personal_data['BIRTHDAY'].apply(lambda x:datetime.datetime.strptime(x,'%Y.%m.%d') if len(x) == 10 else x)
personal_data['AGE'] = personal_data['BIRTHDAY'].apply(lambda x:round((cur_date-x).days/365) if x != '--' else x)
personal_data['EDU_LEVEL'].replace({'研究生': 10,'大学本科（简称"大学"）':20,'大学专科和专科学校（简称"大专"）':21,
                                    '高中':30,'中等专业学校或中等技术学校':31,'技术学校':32,'初中':40,'小学':50,
                                    '文盲或半文盲':60,'--':np.nan}, inplace = True)
personal_data['MARITAL_STSTE'].replace({'已婚': 1,'未婚':0,'离婚':0,'--':0}, inplace = True)
del personal_data['BIRTHDAY']
personal_data.columns = ['REPORT_ID','性别','婚姻状况','学历','年龄']

<ipython-input-63-327dc10e3e11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal_data['GENDER'] = personal_data['GENDER'].replace({'男性':1,'女性':0,'--':0})
<ipython-input-63-327dc10e3e11>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal_data['BIRTHDAY'] = personal_data['BIRTHDAY'].apply(lambda x:datetime.datetime.strptime(x,'%Y.%m.%d') if len(x) == 10 else x)
<ipython-input-63-327dc10e3e11>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [64]:
personal_data.head()

,REPORT_ID,性别,婚姻状况,学历,年龄
0,2018111300006460120325,0,1,40.0,54
1,2018111300006460216344,1,1,40.0,58
2,2018111300006460324975,1,1,21.0,52
3,2018111300006460360719,1,丧偶,20.0,45
4,2018111300006460611616,1,1,21.0,49


## 16.近5年最大逾期与累计逾期

In [65]:
overdue_record = tbl_per_overdue_record[['REPORT_ID','LASTMONTHS']]
overdue_record = overdue_record.groupby('REPORT_ID').agg({'LASTMONTHS':['max','sum']})
overdue_record = overdue_record['LASTMONTHS'].reset_index()
overdue_record.columns = ['REPORT_ID','近5年已结清最大逾期','近5年已结清累计逾期']
overdue_record = overdue_record.merge(overdue_data_all[['REPORT_ID','近24月累计逾期','近24月最大逾期']],how='outer').fillna(0)
cols = overdue_record.columns
overdue_record['近5年最大逾期'] = overdue_record[cols[cols.str.contains('最大')]].max(axis=1)
overdue_record['近5年累计逾期'] = overdue_record[cols[cols.str.contains('累计')]].sum(axis=1)
overdue_record = overdue_record[['REPORT_ID','近5年最大逾期','近5年累计逾期']]

In [66]:
overdue_record.head()

,REPORT_ID,近5年最大逾期,近5年累计逾期
0,2018111300006460120325,1.0,1.0
1,2018111300006460126410,1.0,4.0
2,2018111300006460216344,1.0,5.0
3,2018111300006460324975,1.0,7.0
4,2018111300006460325386,1.0,5.0


## 17.征信白户

In [67]:
customers = set(main_table_use['REPORT_ID'])
account_count = tbl_per_info_summary[['REPORT_ID','LOAN_CARD_COUNT','LOAN_OVERDUE_COUNT','UNPAIDLOAN_FINANCE_ORG_COUNT','GUAR_COUNT',
                                      'HOUSE_LOAN_COUNT','BUSI_HOUSE_LOAN_COUNT','STANDARD_LOAN_CARD_COUNT','OTHER_LOAN_COUNT']]
account_count = account_count.set_index('REPORT_ID').fillna(0)
white_customers = account_count[(account_count == 0).all(axis=1)].index

In [68]:
# account_count[(account_count == 0).all(axis=1)]

## 18.当前状态

In [69]:
curr_state_1 = tbl_per_loan_info[['REPORT_ID','STATE']]
curr_state_2 = tbl_per_loan_card_info[['REPORT_ID','STATE']]
overdue_loan_ids = set(curr_state_1.loc[curr_state_1['STATE'] == '逾期','REPORT_ID'])
overdue_loan_card_ids = set(curr_state_2.loc[curr_state_1['STATE'] == '逾期','REPORT_ID'])
overdue_ids = overdue_loan_card_ids | overdue_loan_ids

In [70]:
main_table_use['当前逾期'] = 0
main_table_use.loc[main_table_use['REPORT_ID'].isin(overdue_ids),'当前逾期']
main_table_use = reduce(all_merge,[main_table_use,overdue_data_all,record_all,exp_bank_data_all,used_credit,
                                  guar_balance,no_settle_info,his_loan_org_count,personal_data,overdue_record,total_loan_info,
                                   pay_and_loan_ratio,accage_loan_card,accage_count_loan,accage_loan])
main_table_use['征信白户'] = 0
main_table_use.loc[main_table_use['REPORT_ID'].isin(white_customers),'征信白户'] = 1

In [71]:
main_table_use_mr['当前逾期'] = 0
main_table_use_mr.loc[main_table_use_mr['REPORT_ID'].isin(overdue_ids),'当前逾期']
main_table_use_table = main_table_use_mr.drop_duplicates()
main_table_use_mr = reduce(all_merge,[main_table_use_mr,overdue_data_all,record_all,exp_bank_data_all,used_credit,
                                  guar_balance,no_settle_info,his_loan_org_count,personal_data,overdue_record,total_loan_info,
                                   pay_and_loan_ratio,accage_loan_card,accage_count_loan,accage_loan])

main_table_use_mr['征信白户'] = 0
main_table_use_mr.loc[main_table_use_mr['REPORT_ID'].isin(white_customers),'征信白户'] = 1

In [72]:
main_table_use_mr = main_table_use_mr.add_prefix('mr_')

In [73]:
main_table_use_mr.rename(columns={'mr_证件号码':'配偶证件号码','mr_配偶证件号码':'证件号码','mr_基准时间':'基准时间'},inplace=True)
main_table_use_mr.drop(['mr_配偶客编'],axis=1,inplace=True)

In [74]:
main_table_all = main_table_use.merge(main_table_use_mr,on=['证件号码','基准时间'],how='left').drop([
                                                                           '配偶证件号码_x','REPORT_ID','REPORTDATE','QUERY_REASON',
                                                                           'date_delta','mr_REPORT_ID','mr_REPORTDATE','mr_QUERY_REASON',
                                                                           'mr_date_delta'],axis=1)

In [75]:
dy = pd.read_excel('信贷数据调阅范围.xlsx')
dy = dy.drop_duplicates()

In [76]:
main_table_all = dy[['证件号码','客户编号']].merge(main_table_all,on=['证件号码'],how='right').drop_duplicates()

In [77]:
main_table_all.to_excel('征信特征汇总2.xlsx',index=False)